In [ ]:
import sys
import os
dir = os.path.abspath('')
while not dir.endswith('ardt'): dir = os.path.dirname(dir)
if not dir in sys.path: sys.path.append(dir)

In [ ]:
from datasets import load_dataset, load_from_disk, concatenate_datasets
from huggingface_hub import login

from utils.helpers import find_root_dir

import numpy as np
import seaborn as sns

In [ ]:
ARDT_DIR = find_root_dir()

In [ ]:
dataset_names = ["arrl_nrmdp_train_halfcheetah_v0", "arrl_nrmdp_train_halfcheetah_v1"]
weight_combs = [
    [1, 1]
]

In [ ]:
def compute_sum(ds):
    return {'returns': sum(ds['rewards'])}

for i, weights in enumerate(weight_combs):    
    super_dataset = None
    for dataset_name in dataset_names:
        # load dataset
        ds = load_from_disk(f"{ARDT_DIR}/datasets/{dataset_name}")
        ds = ds.map(compute_sum)
        # # visualise selection of dataset
        # returns = (np.array(ds['rewards'])).sum(axis=1)
        # sns.displot(returns, kind="kde", bw_adjust=0.5);
        # add to super dataset
        if super_dataset is None:
            super_dataset = ds
        else:
            ds = ds.cast(super_dataset.features)
            super_dataset = concatenate_datasets([super_dataset, ds])
    # visualise combined dataset
    ds_to_vis = super_dataset.map(compute_sum)
    print(f"Datasets: {dataset_names} \n Weights: {weights}")
    # save
    super_dataset.save_to_disk(f'{ARDT_DIR}/datasets/arrl_nrmdp_train_halfcheetah_v6')